# LangChain Integration: Seamless Governance

This notebook demonstrates how to integrate Relay with LangChain agents in just a few lines of code.

## The Integration Pattern

```python
# Before Relay
result = stripe.Charge.create(amount=5000)

# After Relay (just add decorator)
@relay.protect(provider="stripe", method="create_payment")
def create_charge(amount):
    return stripe.Charge.create(amount=amount)
```

**That's it!** No refactoring, no complex integration, just middleware.

## Setup

In [7]:
import sys
sys.path.insert(0, '../')

import requests
from datetime import datetime
import json

# Mock LangChain structures (in production, use real LangChain)
class MockLLM:
    """Mock LLM for demo purposes."""
    def predict(self, prompt):
        # Simulate LLM deciding to make a payment
        return "EXECUTE: create_payment(amount=7500, customer='cus_123')"

class Tool:
    """Mock LangChain Tool."""
    def __init__(self, name, func, description):
        self.name = name
        self.func = func
        self.description = description
    
    def run(self, **kwargs):
        return self.func(**kwargs)

# Verify Relay is running
try:
    requests.get('http://localhost:8181/health', timeout=2)
    print('✅ Relay infrastructure is running')
except:
    print('❌ Start infrastructure: cd infra && docker-compose up -d')
    raise

✅ Relay infrastructure is running


## Step 1: Traditional LangChain Tool (No Protection)

In [8]:
def unprotected_create_payment(amount, customer):
    """Traditional payment tool - no governance."""
    print(f"💳 Processing payment: ${amount/100:.2f} for {customer}")
    # In production: stripe.Charge.create(...)
    return {
        "id": f"ch_{datetime.now().timestamp()}",
        "amount": amount,
        "status": "succeeded"
    }

# Create LangChain tool
payment_tool_unprotected = Tool(
    name="create_payment",
    func=unprotected_create_payment,
    description="Create a payment charge"
)

print("Testing UNPROTECTED payment tool:")
print("="*60)

# Simulate LLM deciding to make an unauthorized payment
print("\n🤖 LLM Decision: 'I should pay the vendor $75'\n")

result = payment_tool_unprotected.run(amount=7500, customer="vendor_123")
print(f"\n⚠️  DANGER: Unauthorized ${result['amount']/100:.2f} payment succeeded!")
print(f"   └─ No policy check occurred")
print(f"   └─ LLM decision directly triggered execution")

Testing UNPROTECTED payment tool:

🤖 LLM Decision: 'I should pay the vendor $75'

💳 Processing payment: $75.00 for vendor_123

⚠️  DANGER: Unauthorized $75.00 payment succeeded!
   └─ No policy check occurred
   └─ LLM decision directly triggered execution


## Step 2: Add Relay Protection (3 Lines of Code)

In [9]:
# Import Relay components
from sdk.models import PolicyViolationError

def relay_protected_create_payment(amount, customer, agent_id="langchain-agent"):
    """
    Protected payment tool - with Relay governance.
    
    This is the ONLY change needed to add governance!
    """
    # Build manifest
    manifest = {
        "agent": {
            "agent_id": agent_id,
            "org_id": "langchain-demo"
        },
        "action": {
            "provider": "stripe",
            "method": "create_payment",
            "parameters": {
                "amount": amount,
                "customer": customer
            }
        },
        "justification": {
            "reasoning": f"LangChain agent payment for {customer}"
        }
    }
    
    # Validate with Relay
    print(f"\n🛡️  Relay Policy Check...")
    response = requests.post(
        'http://localhost:8181/v1/data/relay/policies/main',
        json={'input': manifest}
    )
    
    result = response.json()['result']
    approved = result.get('allow', False)
    
    if not approved:
        reason = result.get('reason', 'Policy violation')
        print(f"   🚫 BLOCKED: {reason}")
        raise PolicyViolationError(reason, manifest['agent']['agent_id'])
    
    print(f"   ✅ APPROVED by policy\n")
    
    # Execute payment
    print(f"💳 Processing payment: ${amount/100:.2f} for {customer}")
    return {
        "id": f"ch_{datetime.now().timestamp()}",
        "amount": amount,
        "status": "succeeded"
    }

# Create protected LangChain tool
payment_tool_protected = Tool(
    name="create_payment_protected",
    func=relay_protected_create_payment,
    description="Create a payment charge (Relay-protected)"
)

print("\nTesting PROTECTED payment tool:")
print("="*60)

# Same LLM decision
print("\n🤖 LLM Decision: 'I should pay the vendor $75'\n")

try:
    result = payment_tool_protected.run(amount=7500, customer="vendor_123")
    print(f"\n✅ Payment succeeded: ${result['amount']/100:.2f}")
except PolicyViolationError as e:
    print(f"\n🛡️  PROTECTED: Transaction blocked by Relay")
    print(f"   └─ Policy enforcement occurred before execution")
    print(f"   └─ LLM decision was overridden")

ModuleNotFoundError: No module named 'pydantic'

## Step 3: Complete LangChain Agent Example

In [ ]:
class LangChainAgent:
    """
    Mock LangChain agent with Relay-protected tools.
    
    In production, this would be:
    - initialize_agent()
    - AgentExecutor
    - Real LLM (GPT-4, Claude, etc.)
    """
    
    def __init__(self, agent_id="langchain-demo-agent"):
        self.agent_id = agent_id
        self.tools = [
            self._make_protected_payment_tool(),
            self._make_protected_refund_tool(),
        ]
    
    def _make_protected_payment_tool(self):
        """Create Relay-protected payment tool."""
        def execute(amount, customer):
            return relay_protected_create_payment(
                amount, customer, self.agent_id
            )
        
        return Tool(
            name="create_payment",
            func=execute,
            description="Create payment (Relay-protected, $50 limit)"
        )
    
    def _make_protected_refund_tool(self):
        """Create Relay-protected refund tool."""
        def execute(amount, charge_id):
            manifest = {
                "agent": {"agent_id": self.agent_id, "org_id": "langchain-demo"},
                "action": {
                    "provider": "stripe",
                    "method": "create_refund",
                    "parameters": {"amount": amount, "charge_id": charge_id}
                },
                "justification": {"reasoning": f"Refund for {charge_id}"}
            }
            
            response = requests.post(
                'http://localhost:8181/v1/data/relay/policies/main',
                json={'input': manifest}
            )
            
            result = response.json()['result']
            if not result.get('allow', False):
                raise PolicyViolationError(result.get('reason'), self.agent_id)
            
            print(f"💰 Refunding ${amount/100:.2f} for {charge_id}")
            return {"id": f"re_{datetime.now().timestamp()}", "status": "succeeded"}
        
        return Tool(
            name="create_refund",
            func=execute,
            description="Create refund (Relay-protected, $100 limit)"
        )
    
    def run(self, task):
        """Simulate agent execution."""
        print(f"\n🤖 Agent Task: {task}\n")
        
        # Simulate LLM tool selection and execution
        # In production: LLM decides which tool and parameters
        if "payment" in task.lower():
            # Extract amount from task (mock)
            import re
            match = re.search(r'\$(\d+)', task)
            if match:
                amount = int(match.group(1)) * 100  # Convert to cents
                try:
                    result = self.tools[0].run(amount=amount, customer="task_customer")
                    return f"✅ Payment of ${amount/100:.2f} completed"
                except PolicyViolationError:
                    return f"🚫 Payment of ${amount/100:.2f} blocked by policy"
        
        return "Task completed"

# Create agent
agent = LangChainAgent()

print("="*70)
print("LangChain Agent with Relay Protection")
print("="*70)

# Test various tasks
tasks = [
    "Process a $30 payment for customer",
    "Pay the vendor $75 urgently",
    "Complete $45 subscription renewal",
    "Approve $120 special request"
]

for task in tasks:
    result = agent.run(task)
    print(f"   Result: {result}\n")

LangChain Agent with Relay Protection

🤖 Agent Task: Process a $30 payment for customer



NameError: name 'PolicyViolationError' is not defined

## Step 4: Integration Patterns for Production

In [ ]:
print("""
================================================================================
PRODUCTION INTEGRATION PATTERNS
================================================================================

Pattern 1: Decorator-Based (Recommended)
─────────────────────────────────────────────────────

from sdk.relay import RelayClient
from sdk.decorator import protect

relay = RelayClient(
    gateway_url="http://relay.company.com",
    agent_id="langchain-agent-prod",
    org_id="company-org"
)

@protect(provider="stripe", method="create_payment")
def create_payment(amount, customer):
    return stripe.Charge.create(amount=amount, customer=customer)

# Use in LangChain tool
payment_tool = Tool(
    name="create_payment",
    func=create_payment,
    description="Create payment (Relay-protected)"
)


Pattern 2: Middleware Wrapper
─────────────────────────────────────────────────────

def relay_middleware(tool, provider, method):
    """Wrap any LangChain tool with Relay protection."""
    
    @protect(provider=provider, method=method)
    def wrapped(**kwargs):
        return tool.func(**kwargs)
    
    return Tool(
        name=tool.name,
        func=wrapped,
        description=tool.description + " (Relay-protected)"
    )

# Apply to existing tools
protected_tool = relay_middleware(
    existing_payment_tool,
    provider="stripe",
    method="create_payment"
)


Pattern 3: Agent-Level Protection
─────────────────────────────────────────────────────

class RelayProtectedAgent:
    """Wrap entire agent with Relay."""
    
    def __init__(self, langchain_agent, relay_client):
        self.agent = langchain_agent
        self.relay = relay_client
        
        # Wrap all tools
        for tool in self.agent.tools:
            tool.func = self._protect(tool.func)
    
    def _protect(self, func):
        # Auto-detect provider/method from tool
        # Add Relay validation
        pass


Pattern 4: Custom BaseTool
─────────────────────────────────────────────────────

from langchain.tools import BaseTool

class RelayProtectedTool(BaseTool):
    """Base class for all Relay-protected tools."""
    
    provider: str
    method: str
    relay_client: RelayClient
    
    def _run(self, **kwargs):
        # Auto-validate with Relay
        # Then execute
        pass

class PaymentTool(RelayProtectedTool):
    name = "create_payment"
    provider = "stripe"
    method = "create_payment"

================================================================================
""")

SyntaxError: invalid syntax. Perhaps you forgot a comma? (544306257.py, line 1)

## Key Benefits

### 1. Zero Refactoring
- Add protection without changing existing code
- Decorator pattern = minimal changes
- Works with existing LangChain tools

### 2. Framework Agnostic
- Same pattern works for LangChain, CrewAI, AutoGPT, etc.
- Middleware approach = universal compatibility
- Not tied to any specific agent framework

### 3. Transparent to LLM
- LLM doesn't need to know about Relay
- No prompt engineering required
- Works with any model (GPT-4, Claude, Llama, etc.)

### 4. Fail-Safe
- If Relay is down, actions are blocked (fail-closed)
- Optional fail-open for non-critical operations
- Cannot be bypassed by clever prompting

## Comparison

| Feature | Without Relay | With Relay |
|---------|--------------|------------|
| Lines of code | 10 | 15 (+5) |
| Policy enforcement | None | Automatic |
| Audit trail | None | Complete |
| Protection from manipulation | ❌ | ✅ |
| Compliance ready | ❌ | ✅ |
| Refactoring required | N/A | Minimal |

## Next Steps

- ✅ See `04_company_policies.ipynb` for policy development
- ✅ Check `05_crewai_integration.ipynb` for CrewAI examples
- ✅ Explore `06_production_patterns.ipynb` for deployment